In [ ]:
# Clone the repository and install the necessary packages
!git clone https://github.com/JakubMartinka/karlsruhe2024.git
!pip3 install mlatom
!pip3 install py3Dmol

In [ ]:
#Import MLatom as a Python package.
import mlatom as ml

In [ ]:
#First, we define a MS-ANI model
model = ml.models.msani(model_file='fulvene_tutorial.npz',nstates=2)

In [ ]:
#then, we load the training data as a MLatom molecular database. This is a small dataset for
#demo purposes.
train_data = ml.data.molecular_database.load("karlsruhe2024/materials/tutorial_data.json", format="json")

In [ ]:
#Now we can train the model.
model.train(molecular_database=train_data,
            property_to_learn='energy',
            xyz_derivative_property_to_learn='energy_gradients',
            hyperparameters={'max_epochs': 100}) #100 epochs is not enough, only for demo.

In [ ]:
#The model can be used to make single-point predictions
#First we need to load a fulvene molecule from .xyz
mol = ml.data.molecule()
mol.read_from_xyz_file('karlsruhe2024/materials/fulvene.xyz')
mol.view()

In [ ]:
#Now we can make a prediction
model.predict(molecule=mol, nstates=2, current_state=0,
              calculate_energy=True, calculate_energy_gradients=True)

#And print the results
for state in mol.electronic_states:
    print(state.energy)
gap = (mol.electronic_states[1].energy-mol.electronic_states[0].energy)*ml.constants.Hartree2eV
print("The S1-S0 gap is {} eV".format(gap))

In [ ]:
#Finally, we can use the MS-ANI model to run NAMD.
#For that we will load a model trained in the AL loop.
model_namd = ml.models.msani(model_file='karlsruhe2024/materials/fulvene_energy_iteration_19.npz')

In [ ]:
#Load an initial conditions database
init_db = ml.data.molecular_database.load("karlsruhe2024/materials/init_cond_db.json", format="json")
#And define NAMD arguments
timemax = 60 # fs
namd_kwargs = {
            'model': model_namd,
            'time_step': 0.1, # fs
            'maximum_propagation_time': timemax,
            'hopping_algorithm': 'LZBL',
            'nstates': 2,
            'reduce_kinetic_energy': True,
            }

In [ ]:
#And finally run the NAMD. This should take about a minute.
dyns = ml.simulations.run_in_parallel(molecular_database=init_db[:4], task=ml.namd.surface_hopping_md, task_kwargs=namd_kwargs, create_and_keep_temp_directories=False)


In [ ]:
#Now we can plot the results
trajs = [d.molecular_trajectory for d in dyns]
ml.namd.plot_population(trajectories=trajs, time_step=0.1,
                        max_propagation_time=timemax, nstates=2)

In [ ]:
#MLatom trajectories can be analyzed and visualized using simple Python code. An example below.
energies = [[],[]]
time = []
gap = []
for step in trajs[0].steps:
    energies[0].append(step.molecule.electronic_states[0].energy)
    energies[1].append(step.molecule.electronic_states[1].energy)
    time.append(step.time)
    gap.append(step.molecule.electronic_states[1].energy-step.molecule.electronic_states[0].energy)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(time, np.transpose(energies))
plt.xlabel("Time (fs)")
plt.ylabel("Energy (Hartree)")
plt.legend(['S0', 'S1'])


In [ ]:
trajs[0].view()